In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing the libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

## Encode categorical data

In [6]:
dataset = pd.read_csv('/content/drive/My Drive/chat_dataset.csv')
y = dataset.iloc[:, -1].values
X = dataset.iloc[:, :-1].values


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

dataset.iloc[:, -1] = y_encoded

dataset.to_csv('/content/drive/My Drive/chat_dataset_encoded.csv', index=False)


<ipython-input-6-c5b6615c8be3>:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dataset.iloc[:, -1] = y_encoded


## Split csv's into training set, test set and validation set

In [7]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/drive/My Drive/chat_dataset_encoded.csv')

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(len(train_df), len(test_df), len(validation_df))

train_df.to_csv('/content/drive/My Drive/train_dataset.csv', index=False)
validation_df.to_csv('/content/drive/My Drive/validation_dataset.csv', index=False)
test_df.to_csv('/content/drive/My Drive/test_dataset.csv', index=False)

408 88 88


## Importing the dataset

In [25]:
train_dataset = tf.data.experimental.make_csv_dataset(
    '/content/drive/My Drive/train_dataset.csv',
    batch_size=50,
    column_names = ['keras_layer_3_input', 'sentiment'],
    label_name='sentiment',
    na_value="?",
    num_epochs=1,
    shuffle=True,
    ignore_errors=True)

validation_dataset = tf.data.experimental.make_csv_dataset(
    '/content/drive/My Drive/validation_dataset.csv',
    batch_size=10,
    column_names = ['keras_layer_3_input', 'sentiment'],
    label_name='sentiment',
    na_value="?",
    num_epochs=1,
    ignore_errors=True)

test_dataset = tf.data.experimental.make_csv_dataset(
    '/content/drive/My Drive/test_dataset.csv',
    batch_size=10,
    column_names = ['keras_layer_3_input', 'sentiment'],
    label_name='sentiment',
    na_value="?",
    num_epochs=1,
    ignore_errors=True)



## Construct Tensorflow Hub embedding model

In [26]:
import tensorflow_hub as hub


embedding = "https://www.kaggle.com/models/google/nnlm/frameworks/TensorFlow2/variations/en-dim128-with-normalization/versions/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)


## Construct model itself

In [27]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 128)               124642688 
                                                                 
 dense_8 (Dense)             (None, 16)                2064      
                                                                 
 dense_9 (Dense)             (None, 3)                 51        
                                                                 
Total params: 124644803 (475.48 MB)
Trainable params: 124644803 (475.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Compile the model

In [28]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

## Train the model

In [29]:
history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=validation_dataset)

Epoch 1/10
9/9 [==============================] - 53s 6s/step - loss: 1.0860 - accuracy: 0.3799 - val_loss: 1.0744 - val_accuracy: 0.5000
Epoch 2/10
9/9 [==============================] - 36s 4s/step - loss: 1.0148 - accuracy: 0.5760 - val_loss: 1.0214 - val_accuracy: 0.5114
Epoch 3/10
9/9 [==============================] - 36s 4s/step - loss: 0.9505 - accuracy: 0.6250 - val_loss: 0.9655 - val_accuracy: 0.5795
Epoch 4/10
9/9 [==============================] - 37s 4s/step - loss: 0.8807 - accuracy: 0.7132 - val_loss: 0.9089 - val_accuracy: 0.6705
Epoch 5/10
9/9 [==============================] - 37s 4s/step - loss: 0.8035 - accuracy: 0.7892 - val_loss: 0.8488 - val_accuracy: 0.7273
Epoch 6/10
9/9 [==============================] - 43s 5s/step - loss: 0.7235 - accuracy: 0.8309 - val_loss: 0.7870 - val_accuracy: 0.7045
Epoch 7/10
9/9 [==============================] - 37s 4s/step - loss: 0.6409 - accuracy: 0.8554 - val_loss: 0.7247 - val_accuracy: 0.7273
Epoch 8/10
9/9 [==================

## Evaluate the model

In [30]:
results = model.evaluate(test_dataset, verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

9/9 - 5s - loss: 0.5798 - accuracy: 0.8182 - 5s/epoch - 523ms/step
loss: 0.580
accuracy: 0.818


## Save the model

In [32]:
model.save('/content/drive/My Drive/my_first_model')